<a href="https://colab.research.google.com/github/cangokalp/char_rnn_rap_generator/blob/master/rap_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lyricsgenius
import lyricsgenius
import pdb
import pickle
import re
import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import os
import sys

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
# from google.colab import files

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def save(fname, data, extension='pickle'):
    path = '/content/gdrive/My Drive/' + fname + "." + extension

    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load(fname, extension='pickle'):
    path = '/content/gdrive/My Drive/' + fname + "." + extension

    with open(path, 'rb') as f:
        item = pickle.load(f)

    return item


In [15]:
import time
API_TOKEN = '1OWmIr37ZuUI5hDXjN_FIXJlbZadIqH6JBDObBCiF_MNySn4BQqh8aQ8OUYF5muk'
MAX_SONGS = 'mix'

#### grap the lyrics #####
try:
	documents = load('lyrics_array_' + str(MAX_SONGS))

except:
  artists = []
  genius = lyricsgenius.Genius(API_TOKEN, skip_non_songs=True, remove_section_headers=True)

  lonely = genius.search_artist("The Lonely Island", max_songs=80, sort="popularity")
  artists.append(lonely)
  time.sleep(10)

  fetty = genius.search_artist("Fetty Wap", max_songs=15, sort="popularity")
  artists.append(fetty)
  childish = genius.search_artist("Childish Gambino", max_songs=20, sort="popularity")
  artists.append(childish)
  drake = genius.search_artist("Drake", max_songs=20, sort="popularity")
  artists.append(drake)
  kendrick = genius.search_artist("Kendrick Lamar", max_songs=20, sort="popularity")
  artists.append(kendrick)
  time.sleep(20)

  drake = genius.search_artist("Lil Dicky", max_songs=20, sort="popularity")
  artists.append(drake)
  chet = genius.search_artist("Chet Faker", max_songs=10, sort="popularity")
  artists.append(chet)
  snoop = genius.search_artist("Snoop Dogg", max_songs=15, sort="popularity")
  artists.append(snoop)
  travis = genius.search_artist("Travis Scott", max_songs=10, sort="popularity")
  artists.append(travis)
  q = genius.search_artist("ScHoolboy Q", max_songs=10, sort="popularity")
  artists.append(q)
  wayne = genius.search_artist("Lil Wayne", max_songs=15, sort="popularity")
  artists.append(wayne)
  time.sleep(30)

  eminem = genius.search_artist("Eminem", max_songs=10, sort="popularity")
  artists.append(eminem)
  ariana = genius.search_artist("Ariana Grande", max_songs=5, sort="popularity")
  artists.append(ariana)
  wiz = genius.search_artist("Wiz Khalifa", max_songs=10, sort="popularity")
  artists.append(wiz)
  beatles = genius.search_artist("The Beatles", max_songs=15, sort="popularity")
  artists.append(beatles)
  billie = genius.search_artist("Billie Eilish", max_songs=15, sort="popularity")
  artists.append(billie)
  peas = genius.search_artist("Black Eyed Peas", max_songs=10, sort="popularity")
  artists.append(peas)
  time.sleep(40)

  chance = genius.search_artist("Chance The Rapper", max_songs=10, sort="popularity")
  artists.append(chance)
  kanye = genius.search_artist("Kanye West", max_songs=15, sort="popularity")
  artists.append(kanye)
  outkast = genius.search_artist("OutKast", max_songs=10, sort="popularity")
  artists.append(outkast)
  rihanna = genius.search_artist("Rihanna", max_songs=10, sort="popularity")
  artists.append(rihanna)


  documents = []
  for artist in artists:
    
    for song in artist.songs:
      if artist == artists[0] and song == artist.songs[0]:
        continue
      documents.append(song.lyrics)

save('lyrics_array_' + str(MAX_SONGS), documents)

Searching for songs by The Lonely Island...

Song 1: "Spell It Out"
Song 2: "I’m On a Boat"
Song 3: "YOLO"
Song 4: "Jack Sparrow"
Song 5: "Finest Girl (Bin Laden Song)"
Song 6: "I Just Had Sex"
Song 7: "SEMICOLON"
Song 8: "Motherlover"
Song 9: "I’m So Humble"
Song 10: "Dick In a Box"
Song 11: "3 Way (The Golden Rule)"
Song 12: "Threw It On the Ground"
Song 13: "Natalie’s Rap"
Song 14: "I Fucked My Aunt"
Song 15: "Spring Break Anthem"
Song 16: "Jizz In My Pants"
Song 17: "Diaper Money"
Song 18: "Equal Rights"
Song 19: "Like a Boss"
Song 20: "Go Kindergarten"
Song 21: "Lazy Sunday"
Song 22: "Natalie’s Rap 2.0"
Song 23: "Hugs"
Song 24: "Ras Trent"
Song 25: "Mona Lisa"
Song 26: "Shy Ronnie 2: Ronnie & Clyde"
Song 27: "The Creep"
Song 28: "Boombox"
Song 29: "Fuck Off"
Song 30: "Incredible Thoughts"
Song 31: "No Homo"
Song 32: "Hunter the Hungry Is Gon’ Eat"
Song 33: "Turn Up the Beef"
Song 34: "Shy Ronnie"
Song 35: "You’ve Got the Look"
Song 36: "We Like Sportz"
Song 37: "Oakland Nights"
So

AttributeError: ignored

In [0]:

comedy_rap_text = ''
for document in documents:
  if document is not None:
    comedy_rap_text += document + '\n\n'

comedy_rap_prepped = comedy_rap_text
import re
pattern = re.compile(r'[(.?)]')
comedy_rap_prepped = re.sub(pattern, '', comedy_rap_prepped)



In [0]:
comedy_rap_text = comedy_rap_prepped
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(comedy_rap_text)
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count
[encoded] = np.array(tokenizer.texts_to_sequences([comedy_rap_text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=20)

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]


def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

tf.random.set_seed(42)

# text = complete_text("h", temperature=1, n_chars=200)

In [0]:
model.save('/content/gdrive/My Drive/rap_bot_v2_model.h5')

In [0]:
text = complete_text("y", temperature=1, n_chars=1000)

In [0]:
f = open('/content/gdrive/My Drive/bot_rap.txt','w+')
f.write(text)